In [2]:
# Total number of events based on their category 

SELECT c.catgroup, COUNT(e.eventid) AS event_count
FROM category c
LEFT JOIN event e ON c.catid = e.catid
GROUP BY c.catgroup
ORDER BY event_count DESC;

,catgroup,event_count
0,Concerts,4998
1,Shows,3800
2,Sports,0


In [8]:
# Top 10 events based on total revenue generated and their location

SELECT e.eventname, v.venuecity, SUM(s.pricepaid) AS total_revenue
FROM event e
JOIN venue v ON e.venueid = v.venueid
JOIN sales s ON e.eventid = s.eventid
GROUP BY e.eventname, v.venuecity
ORDER BY total_revenue DESC
LIMIT 10;

,eventname,venuecity,total_revenue
0,Spring Awakening,New York City,726824.0
1,The Country Girl,New York City,709949.0
2,Mamma Mia!,New York City,709672.0
3,Jersey Boys,New York City,669345.0
4,Macbeth,New York City,664873.0
5,Mystere Cirque du Soleil,Las Vegas,583283.0
6,Chicago,New York City,582057.0
7,Blue Man Group,Las Vegas,567346.0
8,The Seagull,New York City,527282.0
9,Thurgood,New York City,525173.0


In [9]:
# Top 5 category names based on their average ticket price

SELECT catname, AVG(priceperticket) AS avg_ticket_price
FROM category c
JOIN event e ON c.catid = e.catid
JOIN listing l ON e.eventid = l.eventid
GROUP BY catname
ORDER BY avg_ticket_price DESC
LIMIT 5;

,catname,avg_ticket_price
0,Opera,402.69
1,Plays,398.53
2,Musicals,397.81
3,Pop,396.12


In [31]:
# Top 5 buyers from California who spent the most on concerts

SELECT u.firstname || ' ' || u.lastname AS full_name, SUM(s.pricepaid) AS total_spending_on_concerts
FROM users u
JOIN sales s ON u.userid = s.buyerid
JOIN event e ON s.eventid = e.eventid
JOIN category c ON e.catid = c.catid
JOIN venue v ON e.venueid = v.venueid
WHERE c.catgroup = 'Concerts' AND v.venuestate = 'CA'
GROUP BY full_name
ORDER BY total_spending_on_concerts DESC
LIMIT 5;

,full_name,total_spending_on_concerts
0,Brianna England,21568.0
1,Yoshio Clements,14820.0
2,Fletcher Briggs,14120.0
3,Gisela Huffman,12568.0
4,Willa Conley,10612.0


In [21]:
# Top 5 locations based on their ticket price

SELECT v.venuecity, v.venuestate, ROUND(AVG(l.priceperticket), 0) AS avg_event_price
FROM venue v
JOIN event e ON v.venueid = e.venueid
JOIN listing l ON e.eventid = l.eventid
GROUP BY v.venuecity, v.venuestate
ORDER BY avg_event_price DESC
LIMIT 5;

,venuecity,venuestate,avg_event_price
0,Foxborough,MA,438
1,St. Paul,MN,437
2,Orchard Park,NY,427
3,Mountain View,CA,426
4,Chicago,IL,417


In [24]:
# Top 5 events and their location based on occupancy

WITH event_sales AS (
    SELECT e.eventid, SUM(s.qtysold) AS total_tickets_sold
    FROM sales s
    JOIN event e ON s.eventid = e.eventid
    GROUP BY e.eventid
)

SELECT e.eventname, v.venuename, 
       ROUND(
           CASE WHEN v.venueseats > 0 
                THEN (event_sales.total_tickets_sold / v.venueseats::float) * 100
                ELSE 0 
           END, 2) AS occupancy_percentage
FROM event_sales
JOIN event e ON event_sales.eventid = e.eventid
JOIN venue v ON e.venueid = v.venueid -- Fixed typo here
ORDER BY occupancy_percentage DESC
LIMIT 5;

,eventname,venuename,occupancy_percentage
0,Insane Clown Posse,Madison Square Garden,0.45
1,Foreigner,Shoreline Amphitheatre,0.44
2,Brian Jonestown Massacre,Madison Square Garden,0.43
3,Hermans Hermits,Madison Square Garden,0.37
4,Steve Miller Band,Madison Square Garden,0.36
